In [2]:
!pip install numpy

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)


In [3]:
!pip install pandas

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 435.7 kB/s eta 0:00:27
   ---------------------------------------- 0.0/11.5 MB 326.8 kB/s eta 0:00:36
   ---------------------------------------- 0.1/11.5 MB 409.6 kB/s eta 0:00:28
   ---------------------------------------- 0.1/11.5 MB 416.7 kB/s eta 0:00:28
   ---------------------------------------- 0.1/11.5 MB 467.6 kB/s eta 0:00:25
   ---------------------------------------- 0.1/11.5 MB 450.6 kB/s eta 0:00:26
   ---------------------------------------- 0.1/11.5 MB 473.7 kB/s eta 0:00:24
    --------------------------------------- 0.2/11.5 MB 456.4 kB/s eta 0:00:25
    --------------------------------------- 0.2/11.5 MB 580.1 kB/s eta 0:00:20
    ------------

In [7]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.6 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 645.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 991.0 kB/s eta 0:00:11
   ---------------------------------------- 0.1/10.6 MB 787.7 kB/s eta 0:00:14
   ---------------------------------------- 0.1/10.6 MB 901.1 kB/s eta 0:00:12
    --------------------------------------- 0.2/10.6 MB 838.4 kB/s eta 0:00:13
    --------------------------------------- 0.2/10.6 MB 845.5 kB/s eta 0:00:13
    --------------------------------------- 0.2/10.6 MB 811.5 kB/s eta 0:00:13
   - -------------------------------------- 0.3/10.6 MB 853.3 kB/s eta 0:00:13
   - -------------------------------------- 0.3/10.6 MB 819.2 kB/s eta 0

In [4]:
import numpy as np
import pandas as pd

In [19]:
train_data=pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

In [22]:
train_data.info()
print("*************************")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162758 entries, 0 to 162757
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   movieid             162758 non-null  object
 1   reviewerName        162758 non-null  object
 2   isFrequentReviewer  162758 non-null  bool  
 3   reviewText          156311 non-null  object
 4   sentiment           162758 non-null  object
dtypes: bool(1), object(4)
memory usage: 5.1+ MB
*************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55315 entries, 0 to 55314
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieid       55315 non-null  object
 1   reviewerName  55315 non-null  object
 2   isTopCritic   55315 non-null  bool  
 3   reviewText    52805 non-null  object
dtypes: bool(1), object(3)
memory usage: 1.3+ MB


In [37]:
train_data_filtered=train_data.dropna()
test_data_filtered=test_data.dropna()

In [38]:
X=train_data_filtered.drop(['movieid', 'reviewerName','isFrequentReviewer', 'sentiment'], axis=1)
y=train_data_filtered['sentiment']

In [39]:
X.shape, y.shape

((156311, 1), (156311,))

In [41]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y, test_size=.02,random_state=1)

In [42]:
import re

def normalize(text):
    text=text.lower()
    text=re.sub(r'[^\w\s]', '', text)
    text=re.sub(r'\d', '', text)
    text=re.sub(r'\s+', ' ', text)
    text=text.strip()
    return text 

In [43]:
X_train['reviewText']=X_train['reviewText'].apply(lambda x: normalize(x))
X_test['reviewText']=X_test['reviewText'].apply(lambda x: normalize(x))

In [45]:
X_train[:5], y[:5]

(                                               reviewText
 63393   by bantering with the lads and slapping their ...
 34241   watching it didnt feel like getting sucked int...
 113321  ted bundys psychology is never explored in the...
 108438                    ive never seen anything like it
 62261   in a medium where the important decisions are ...,
 0    POSITIVE
 1    NEGATIVE
 2    POSITIVE
 3    POSITIVE
 4    POSITIVE
 Name: sentiment, dtype: object)

In [47]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
ctf1 = ColumnTransformer(
    transformers=[
        ('tfidf',tfidf , 'reviewText')
    ])

ctf1.fit(X_train)

ColumnTransformer(transformers=[('tfidf', TfidfVectorizer(), 'reviewText')])

In [48]:
X_train_transformed1 = ctf1.transform(X_train)
X_test_transformed1 = ctf1.transform(X_test)
X_train_transformed1.shape,X_test_transformed1.shape

((153184, 88193), (3127, 88193))

In [50]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score,accuracy_score



model=SGDClassifier(loss='modified_huber')
model.fit(X_train_transformed1,y_train)
model_pred=model.predict(X_test_transformed1)
print("Accuracy_score:",accuracy_score(model_pred,y_test),"F1_score:",f1_score(model_pred,y_test,pos_label='POSITIVE'))

Accuracy_score: 0.8062040294211704 F1_score: 0.8644295302013423
